# Prepare dataset

- Create dataset from the new dataframe

## Librairies

In [ ]:
# pytorch version '1.9.0a0+df837d0'

In [1]:
!pip install pandas==1.3.4
!pip install transformers==4.12.5
!pip install datasets==1.15.1

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import os
import pickle

import pandas as pd

import numpy as np
import torch

import transformers
from transformers import BertTokenizer

import datasets
from datasets import concatenate_datasets
from datasets import Dataset
from datasets import ClassLabel
from datasets import DatasetDict

In [3]:
print('pandas:\t\t', pd.__version__)
print('transformers:\t', transformers.__version__)
print('datasets:\t', datasets.__version__)

pandas:		 1.3.4
transformers:	 4.12.5
datasets:	 1.15.1


## Global variables

In [4]:
DATA_FOLDER = '/notebooks/link_identification_task/data'
DATA_FILE = '/notebooks/link_identification_task/data/pe_w_strct_fts_comp_type.csv'
# FEATURES_1_FILE = '/raid/home/jeremiec/Data/persuasive_essays/features'
# FEATURES_2_FILE = '/raid/home/jeremiec/Data/persuasive_essays/features_2'

In [5]:
print(DATA_FILE)

/notebooks/link_identification_task/data/pe_w_strct_fts_comp_type.csv


In [6]:
dataset_df = pd.read_csv(DATA_FILE)

In [7]:
dataset_df.head(13)

,Unnamed: 0,essay_nr,starting_idx,component_label,ending_idx,text,link_label,split,essay,argument_bound_1,argument_bound_2,argument_id,label,structural_fts_as_text,joined_label,strct_fts_w_linked,strct_fts_w_comp_type
0,0,essay001,503,MajorClaim,575,we should attach more importance to cooperatio...,not_linked,TRAIN,Should students be taught to compete or to coo...,503,575,1,MajorClaim,Topic: Should students be taught to compete or...,MC-NL,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...
1,1,essay001,591,Claim,714,"through cooperation, children can learn about ...",linked,TRAIN,Should students be taught to compete or to coo...,591,714,2,Claim,Topic: Should students be taught to compete or...,CL-L,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...
2,2,essay001,716,Premise,851,What we acquired from team work is not only ho...,not_linked,TRAIN,Should students be taught to compete or to coo...,716,851,3,Premise,Topic: Should students be taught to compete or...,PREM-NL,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...
3,3,essay001,853,Premise,1086,"During the process of cooperation, children ca...",not_linked,TRAIN,Should students be taught to compete or to coo...,853,1086,4,Premise,Topic: Should students be taught to compete or...,PREM-NL,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...
4,4,essay001,1088,Premise,1191,All of these skills help them to get on well w...,not_linked,TRAIN,Should students be taught to compete or to coo...,1088,1191,5,Premise,Topic: Should students be taught to compete or...,PREM-NL,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...
5,5,essay001,1212,Premise,1301,the significance of competition is that how to...,not_linked,TRAIN,Should students be taught to compete or to coo...,1212,1301,6,Premise,Topic: Should students be taught to compete or...,PREM-NL,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...
6,6,essay001,1332,Claim,1376,competition makes the society more effective,linked,TRAIN,Should students be taught to compete or to coo...,1332,1376,7,Claim,Topic: Should students be taught to compete or...,CL-L,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...
7,7,essay001,1387,Premise,1492,when we consider about the question that how t...,not_linked,TRAIN,Should students be taught to compete or to coo...,1387,1492,8,Premise,Topic: Should students be taught to compete or...,PREM-NL,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...
8,8,essay001,1549,Premise,1846,Take Olympic games which is a form of competit...,not_linked,TRAIN,Should students be taught to compete or to coo...,1549,1846,9,Premise,Topic: Should students be taught to compete or...,PREM-NL,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...
9,9,essay001,1927,Claim,1992,"without the cooperation, there would be no vic...",linked,TRAIN,Should students be taught to compete or to coo...,1927,1992,10,Claim,Topic: Should students be taught to compete or...,CL-L,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...


## Prepare dataframe

In [8]:
dataset_df = dataset_df.drop(dataset_df.columns[0], axis=1)

In [9]:
dataset_df = dataset_df.rename(columns={"link_label": "labels"})

In [10]:
dataset_df

,essay_nr,starting_idx,component_label,ending_idx,text,labels,split,essay,argument_bound_1,argument_bound_2,argument_id,label,structural_fts_as_text,joined_label,strct_fts_w_linked,strct_fts_w_comp_type
0,essay001,503,MajorClaim,575,we should attach more importance to cooperatio...,not_linked,TRAIN,Should students be taught to compete or to coo...,503,575,1,MajorClaim,Topic: Should students be taught to compete or...,MC-NL,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...
1,essay001,591,Claim,714,"through cooperation, children can learn about ...",linked,TRAIN,Should students be taught to compete or to coo...,591,714,2,Claim,Topic: Should students be taught to compete or...,CL-L,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...
2,essay001,716,Premise,851,What we acquired from team work is not only ho...,not_linked,TRAIN,Should students be taught to compete or to coo...,716,851,3,Premise,Topic: Should students be taught to compete or...,PREM-NL,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...
3,essay001,853,Premise,1086,"During the process of cooperation, children ca...",not_linked,TRAIN,Should students be taught to compete or to coo...,853,1086,4,Premise,Topic: Should students be taught to compete or...,PREM-NL,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...
4,essay001,1088,Premise,1191,All of these skills help them to get on well w...,not_linked,TRAIN,Should students be taught to compete or to coo...,1088,1191,5,Premise,Topic: Should students be taught to compete or...,PREM-NL,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5845,essay402,1275,Premise,1339,indirectly they will learn how to socialize ea...,not_linked,TRAIN,Children should studying hard or playing sport...,1275,1339,11,Premise,Topic: Children should studying hard or playin...,PREM-NL,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...
5846,essay402,1341,Premise,1388,That will make children getting lots of friends,not_linked,TRAIN,Children should studying hard or playing sport...,1341,1388,12,Premise,Topic: Children should studying hard or playin...,PREM-NL,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...
5847,essay402,1393,Premise,1436,they can contribute positively to community,linked,TRAIN,Children should studying hard or playing sport...,1393,1436,13,Premise,Topic: Children should studying hard or playin...,PREM-L,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...
5848,essay402,1448,Premise,1525,playing sport makes children getting healthy a...,not_linked,TRAIN,Children should studying hard or playing sport...,1448,1525,14,Premise,Topic: Children should studying hard or playin...,PREM-NL,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...


In [11]:
# sanity check
print(len(dataset_df))
dataset_df = dataset_df.dropna()
print(len(dataset_df))

5850
5850


In [12]:
dataset_df = dataset_df.reset_index(drop=True)

In [13]:
train_df = dataset_df[dataset_df.split=='TRAIN'].reset_index(drop=True)
test_df = dataset_df[dataset_df.split=='TEST'].reset_index(drop=True)

# for example

# val_df = dataset_df[dataset_df.split=='VALIDATION'].reset_index(drop=True)

In [14]:
len(train_df), len(test_df)

(4712, 1138)

In [15]:
train_df

,essay_nr,starting_idx,component_label,ending_idx,text,labels,split,essay,argument_bound_1,argument_bound_2,argument_id,label,structural_fts_as_text,joined_label,strct_fts_w_linked,strct_fts_w_comp_type
0,essay001,503,MajorClaim,575,we should attach more importance to cooperatio...,not_linked,TRAIN,Should students be taught to compete or to coo...,503,575,1,MajorClaim,Topic: Should students be taught to compete or...,MC-NL,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...
1,essay001,591,Claim,714,"through cooperation, children can learn about ...",linked,TRAIN,Should students be taught to compete or to coo...,591,714,2,Claim,Topic: Should students be taught to compete or...,CL-L,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...
2,essay001,716,Premise,851,What we acquired from team work is not only ho...,not_linked,TRAIN,Should students be taught to compete or to coo...,716,851,3,Premise,Topic: Should students be taught to compete or...,PREM-NL,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...
3,essay001,853,Premise,1086,"During the process of cooperation, children ca...",not_linked,TRAIN,Should students be taught to compete or to coo...,853,1086,4,Premise,Topic: Should students be taught to compete or...,PREM-NL,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...
4,essay001,1088,Premise,1191,All of these skills help them to get on well w...,not_linked,TRAIN,Should students be taught to compete or to coo...,1088,1191,5,Premise,Topic: Should students be taught to compete or...,PREM-NL,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4707,essay402,1275,Premise,1339,indirectly they will learn how to socialize ea...,not_linked,TRAIN,Children should studying hard or playing sport...,1275,1339,11,Premise,Topic: Children should studying hard or playin...,PREM-NL,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...
4708,essay402,1341,Premise,1388,That will make children getting lots of friends,not_linked,TRAIN,Children should studying hard or playing sport...,1341,1388,12,Premise,Topic: Children should studying hard or playin...,PREM-NL,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...
4709,essay402,1393,Premise,1436,they can contribute positively to community,linked,TRAIN,Children should studying hard or playing sport...,1393,1436,13,Premise,Topic: Children should studying hard or playin...,PREM-L,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...
4710,essay402,1448,Premise,1525,playing sport makes children getting healthy a...,not_linked,TRAIN,Children should studying hard or playing sport...,1448,1525,14,Premise,Topic: Children should studying hard or playin...,PREM-NL,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...


## Prepare Dataset object

In [16]:
dataset_train = Dataset.from_pandas(train_df)
dataset_test = Dataset.from_pandas(test_df)

In [18]:
dataset_train['structural_fts_as_text'][0]

'Topic: Should students be taught to compete or to cooperate?, Sentence: From this point of view, I firmly believe that we should attach more importance to cooperation during primary education., Para Number: 1, First in Para: Yes, Last in Para: Yes, Is in Introduction: Yes, Is in Conclusion: No'

In [19]:
dataset = DatasetDict({"train": dataset_train, "test": dataset_test})

In [20]:
dataset

DatasetDict({
    train: Dataset({
        features: ['essay_nr', 'starting_idx', 'component_label', 'ending_idx', 'text', 'labels', 'split', 'essay', 'argument_bound_1', 'argument_bound_2', 'argument_id', 'label', 'structural_fts_as_text', 'joined_label', 'strct_fts_w_linked', 'strct_fts_w_comp_type'],
        num_rows: 4712
    })
    test: Dataset({
        features: ['essay_nr', 'starting_idx', 'component_label', 'ending_idx', 'text', 'labels', 'split', 'essay', 'argument_bound_1', 'argument_bound_2', 'argument_id', 'label', 'structural_fts_as_text', 'joined_label', 'strct_fts_w_linked', 'strct_fts_w_comp_type'],
        num_rows: 1138
    })
})

In [21]:
# sanity check
set(dataset['train']['split']), set(dataset['test']['split'])

({'TRAIN'}, {'TEST'})

In [22]:
dataset['train']['structural_fts_as_text'][0]

'Topic: Should students be taught to compete or to cooperate?, Sentence: From this point of view, I firmly believe that we should attach more importance to cooperation during primary education., Para Number: 1, First in Para: Yes, Last in Para: Yes, Is in Introduction: Yes, Is in Conclusion: No'

## Save dataset

In [19]:
torch.save(dataset, os.path.join(DATA_FOLDER, 'pe_dataset_w_strct_comp_type.pt'))